# AWS RDS Oracle Database Memory Store for AI Agents

This notebook demonstrates AWS-specific configurations and integrations for deploying Oracle Agent Memory on AWS.

## Features
- **AWS RDS Oracle connectivity** with endpoint management
- **AWS IAM database authentication** (token-based, 15-minute expiry)
- **AWS Secrets Manager** for credential management
- **VPC and security group** support
- **CloudWatch integration** for monitoring and metrics logging
- **AWS Lambda deployment** ready handlers
- **Environment variable configuration**
- **Production-ready patterns** for AWS Lambda, ECS, EC2

## Architecture
This implementation extends the base Oracle Agent Memory to support:
- RDS instance discovery and management
- Secure credential retrieval from Secrets Manager
- IAM-based authentication tokens
- Custom CloudWatch metrics
- Lambda function handlers for serverless deployment

## 1. Install Required Dependencies

In [ ]:
%%capture
import subprocess
import sys

# Install required packages
packages_to_install = [
    "boto3>=1.28.0",
    "botocore>=1.31.0",
    "sqlalchemy>=2.0.0",
    "oracledb>=1.4.0",
    "langchain>=0.0.300",
]

print("Installing required packages...")
for package in packages_to_install:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    
print("✓ All packages installed successfully!")

## 2. Import Libraries and Setup

In [ ]:
import json
import os
 from typing import Dict, Optional, Tuple
from datetime import datetime, timedelta
import boto3
from botocore.exceptions import ClientError

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("✓ All imports successful!")

## 3. AWS RDS Oracle Configuration

Define the AWS RDS Oracle-specific configuration class.

In [ ]:
# ============================================================================
# AWS RDS Oracle Configuration
# ============================================================================

class AWSRDSOracleConfig:
    """AWS RDS Oracle-specific configuration"""
    
    def __init__(
        self,
        db_instance_identifier: str,
        db_endpoint: str,
        port: int = 1521,
        db_name: str = "ORCL",
        aws_region: str = "us-east-1",
        use_iam_auth: bool = True,
    ):
        """
        Initialize AWS RDS Oracle configuration.
        
        Args:
            db_instance_identifier: RDS instance identifier (e.g., 'oracle-prod-db')
            db_endpoint: RDS endpoint hostname
            port: Database port (default 1521 for Oracle)
            db_name: Database name/SID (default ORCL)
            aws_region: AWS region
            use_iam_auth: Whether to use IAM database authentication
        """
        self.db_instance_identifier = db_instance_identifier
        self.db_endpoint = db_endpoint
        self.port = port
        self.db_name = db_name
        self.aws_region = aws_region
        self.use_iam_auth = use_iam_auth
        self.rds_client = boto3.client("rds", region_name=aws_region)
        self.secretsmanager_client = boto3.client("secretsmanager", region_name=aws_region)
    
    def get_rds_instance_info(self) -> Dict:
        """Retrieve RDS instance information from AWS"""
        try:
            response = self.rds_client.describe_db_instances(
                DBInstanceIdentifier=self.db_instance_identifier
            )
            instance = response["DBInstances"][0]
            return {
                "endpoint": instance["Endpoint"]["Address"],
                "port": instance["Endpoint"]["Port"],
                "status": instance["DBInstanceStatus"],
                "engine": instance["Engine"],
                "engine_version": instance["EngineVersion"],
                "allocated_storage": instance.get("AllocatedStorage"),
                "db_instance_class": instance["DBInstanceClass"],
            }
        except ClientError as e:
            print(f"Error retrieving RDS instance info: {e}")
            return {}
    
    def get_credentials_from_secrets_manager(self, secret_name: str) -> Tuple[str, str]:
        """Retrieve database credentials from AWS Secrets Manager"""
        try:
            response = self.secretsmanager_client.get_secret_value(SecretId=secret_name)
            
            if "SecretString" in response:
                secret = json.loads(response["SecretString"])
                username = secret.get("username")
                password = secret.get("password")
                return username, password
            else:
                raise ValueError("Secret is binary, expected JSON format")
        except ClientError as e:
            print(f"Error retrieving secret from Secrets Manager: {e}")
            raise
    
    def create_iam_auth_token(self, db_user: str) -> str:
        """Generate AWS IAM database authentication token (valid for 15 minutes)"""
        try:
            token = self.rds_client.generate_db_auth_token(
                DBHostname=self.db_endpoint,
                Port=self.port,
                DBUser=db_user,
                Region=self.aws_region,
            )
            return token
        except ClientError as e:
            print(f"Error generating IAM auth token: {e}")
            raise

print("✓ AWS RDS Oracle Configuration defined!")

## 4. AWS Environment-Based Configuration

Load configuration from AWS environment variables and Secrets Manager.

In [ ]:
# ============================================================================
# AWS Environment-Based Configuration
# ============================================================================

class AWSEnvironmentConfig:
    """Load configuration from AWS environment variables and Secrets Manager"""
    
    @staticmethod
    def from_environment() -> Tuple[AWSRDSOracleConfig, str, str]:
        """
        Create AWS RDS config from environment variables.
        
        Expected environment variables:
        - AWS_REGION: AWS region
        - RDS_ORACLE_ENDPOINT: RDS hostname
        - RDS_ORACLE_INSTANCE_ID: RDS instance identifier
        - RDS_ORACLE_PORT: Database port (default 1521)
        - RDS_ORACLE_DB_NAME: Database name (default ORCL)
        - RDS_DB_SECRET_NAME: Secrets Manager secret name
        """
        aws_region = os.getenv("AWS_REGION", "us-east-1")
        db_endpoint = os.getenv("RDS_ORACLE_ENDPOINT")
        db_instance_id = os.getenv("RDS_ORACLE_INSTANCE_ID")
        db_port = int(os.getenv("RDS_ORACLE_PORT", "1521"))
        db_name = os.getenv("RDS_ORACLE_DB_NAME", "ORCL")
        secret_name = os.getenv("RDS_DB_SECRET_NAME")
        
        if not db_endpoint:
            raise ValueError("RDS_ORACLE_ENDPOINT environment variable not set")
        if not db_instance_id:
            raise ValueError("RDS_ORACLE_INSTANCE_ID environment variable not set")
        if not secret_name:
            raise ValueError("RDS_DB_SECRET_NAME environment variable not set")
        
        aws_rds_config = AWSRDSOracleConfig(
            db_instance_identifier=db_instance_id,
            db_endpoint=db_endpoint,
            port=db_port,
            db_name=db_name,
            aws_region=aws_region,
        )
        
        # Get credentials from Secrets Manager
        username, password = aws_rds_config.get_credentials_from_secrets_manager(secret_name)
        
        return aws_rds_config, username, password
    
    @staticmethod
    def from_iam_auth() -> Tuple[AWSRDSOracleConfig, str, str]:
        """
        Create AWS RDS config using IAM database authentication.
        
        Expected environment variables:
        - AWS_REGION
        - RDS_ORACLE_ENDPOINT
        - RDS_ORACLE_INSTANCE_ID
        - RDS_DB_IAM_USER: IAM database user
        """
        aws_region = os.getenv("AWS_REGION", "us-east-1")
        db_endpoint = os.getenv("RDS_ORACLE_ENDPOINT")
        db_instance_id = os.getenv("RDS_ORACLE_INSTANCE_ID")
        iam_user = os.getenv("RDS_DB_IAM_USER")
        
        if not all([db_endpoint, db_instance_id, iam_user]):
            raise ValueError("Missing required IAM auth environment variables")
        
        aws_rds_config = AWSRDSOracleConfig(
            db_instance_identifier=db_instance_id,
            db_endpoint=db_endpoint,
            aws_region=aws_region,
            use_iam_auth=True,
        )
        
        # Generate IAM token
        iam_token = aws_rds_config.create_iam_auth_token(iam_user)
        
        return aws_rds_config, iam_user, iam_token

print("✓ AWS Environment Configuration defined!")

## 5. AWS Agent Memory Factory

Factory methods for creating agent memory instances with AWS RDS Oracle.

In [ ]:
# ============================================================================
# AWS Agent Memory Factory
# ============================================================================

class AWSOracleAgentMemoryFactory:
    """Factory for creating agent memory with AWS RDS Oracle"""
    
    @staticmethod
    def create_from_secrets_manager(
        db_instance_identifier: str,
        db_endpoint: str,
        secret_name: str,
        session_id: str,
        agent_id: str = "default_agent",
        aws_region: str = "us-east-1",
    ):
        """
        Create agent memory using credentials from AWS Secrets Manager.
        
        Args:
            db_instance_identifier: RDS instance ID
            db_endpoint: RDS endpoint hostname
            secret_name: Secrets Manager secret name
            session_id: Conversation session ID
            agent_id: Agent identifier
            aws_region: AWS region
        """
        aws_rds_config = AWSRDSOracleConfig(
            db_instance_identifier=db_instance_identifier,
            db_endpoint=db_endpoint,
            aws_region=aws_region,
        )
        
        # Retrieve credentials
        username, password = aws_rds_config.get_credentials_from_secrets_manager(secret_name)
        
        return {
            "config": aws_rds_config,
            "username": username,
            "password": password,
            "session_id": session_id,
            "agent_id": agent_id,
        }
    
    @staticmethod
    def create_from_iam_auth(
        db_instance_identifier: str,
        db_endpoint: str,
        db_user: str,
        session_id: str,
        agent_id: str = "default_agent",
        aws_region: str = "us-east-1",
    ):
        """
        Create agent memory using AWS IAM database authentication.
        
        Args:
            db_instance_identifier: RDS instance ID
            db_endpoint: RDS endpoint hostname
            db_user: IAM database user
            session_id: Conversation session ID
            agent_id: Agent identifier
            aws_region: AWS region
        """
        aws_rds_config = AWSRDSOracleConfig(
            db_instance_identifier=db_instance_identifier,
            db_endpoint=db_endpoint,
            aws_region=aws_region,
            use_iam_auth=True,
        )
        
        # Generate IAM token
        iam_token = aws_rds_config.create_iam_auth_token(db_user)
        
        return {
            "config": aws_rds_config,
            "username": db_user,
            "auth_token": iam_token,
            "session_id": session_id,
            "agent_id": agent_id,
            "use_iam_auth": True,
        }

print("✓ AWS Agent Memory Factory defined!")

## 6. AWS CloudWatch Monitoring

CloudWatch integration for monitoring agent memory operations.

In [ ]:
# ============================================================================
# AWS CloudWatch Monitoring
# ============================================================================

class AWSMemoryMonitoring:
    """CloudWatch monitoring for agent memory operations"""
    
    def __init__(self, aws_region: str = "us-east-1"):
        """Initialize CloudWatch client"""
        self.cloudwatch = boto3.client("cloudwatch", region_name=aws_region)
        self.namespace = "OracleAgentMemory"
        self.metrics_logged = []
    
    def log_metric(
        self,
        metric_name: str,
        value: float,
        unit: str = "Count",
        dimensions: Optional[Dict] = None,
    ) -> None:
        """Log custom metric to CloudWatch"""
        try:
            metric_data = {
                "MetricName": metric_name,
                "Value": value,
                "Unit": unit,
                "Timestamp": datetime.utcnow(),
            }
            
            if dimensions:
                metric_data["Dimensions"] = [
                    {"Name": k, "Value": str(v)} for k, v in dimensions.items()
                ]
            
            # For demonstration, store locally instead of calling CloudWatch
            self.metrics_logged.append(metric_data)
            
        except ClientError as e:
            print(f"Error logging metric to CloudWatch: {e}")
    
    def log_conversation_length(
        self,
        session_id: str,
        message_count: int,
        agent_id: str = "default_agent",
    ) -> None:
        """Log conversation length metric"""
        self.log_metric(
            metric_name="ConversationLength",
            value=message_count,
            dimensions={"SessionId": session_id, "AgentId": agent_id},
        )
    
    def log_db_operation_time(
        self,
        operation_name: str,
        duration_ms: float,
        session_id: str,
    ) -> None:
        """Log database operation duration"""
        self.log_metric(
            metric_name=f"{operation_name}Duration",
            value=duration_ms,
            unit="Milliseconds",
            dimensions={"SessionId": session_id},
        )
    
    def get_logged_metrics(self) -> list:
        """Get all metrics that have been logged"""
        return self.metrics_logged

print("✓ AWS CloudWatch Monitoring defined!")

## 7. AWS Lambda Function Handler

Lambda-compatible handler for AI agents with Oracle memory.

In [ ]:
# ============================================================================
# AWS Lambda Handler
# ============================================================================

class AWSLambdaMemoryHandler:
    """Lambda function handler for AI agent with Oracle memory"""
    
    def __init__(self, aws_region: str = "us-east-1"):
        """Initialize Lambda handler"""
        self.aws_region = aws_region
        self.monitoring = AWSMemoryMonitoring(aws_region)
    
    def create_lambda_handler(self):
        """Create Lambda handler function"""
        def lambda_handler(event, context):
            """
            AWS Lambda handler for AI agent with Oracle memory.
            
            Expected event structure:
            {
                "body": {
                    "message": "User message",
                    "session_id": "user-session-123",
                    "agent_id": "chat_agent"
                }
            }
            """
            try:
                # Parse input
                if isinstance(event.get("body"), str):
                    body = json.loads(event["body"])
                else:
                    body = event.get("body", {})
                
                message = body.get("message")
                session_id = body.get("session_id")
                agent_id = body.get("agent_id", "default_agent")
                
                if not all([message, session_id]):
                    return {
                        "statusCode": 400,
                        "body": json.dumps({"error": "Missing required fields"}),
                    }
                
                # Simulate agent processing
                start_time = datetime.utcnow()
                
                # Log metrics
                duration_ms = (datetime.utcnow() - start_time).total_seconds() * 1000
                self.monitoring.log_db_operation_time(
                    "ProcessMessage",
                    duration_ms,
                    session_id,
                )
                
                return {
                    "statusCode": 200,
                    "body": json.dumps({
                        "message": "Success",
                        "session_id": session_id,
                        "agent_id": agent_id,
                        "processing_time_ms": duration_ms,
                    }),
                }
            
            except Exception as e:
                return {
                    "statusCode": 500,
                    "body": json.dumps({"error": str(e)}),
                }
        
        return lambda_handler

print("✓ AWS Lambda Handler defined!")

---

## 8. Configuration Examples and Testing

Demonstrate AWS RDS Oracle configuration with different authentication methods.

In [ ]:
print("=" * 80)
print("AWS RDS Oracle Agent Memory Examples")
print("=" * 80)

# ========================================================================
# Example 1: Configuration from Secrets Manager
# ========================================================================
print("\n" + "=" * 80)
print("Example 1: Oracle Agent Memory with AWS Secrets Manager")
print("=" * 80)

try:
    memory_config = AWSOracleAgentMemoryFactory.create_from_secrets_manager(
        db_instance_identifier="oracle-prod-db",
        db_endpoint="mydb.xxxxx.us-east-1.rds.amazonaws.com",
        secret_name="rds/oracle/credentials",
        session_id="chat_session_001",
        agent_id="aws_chat_agent",
        aws_region="us-east-1",
    )
    print("✓ Configuration created from Secrets Manager")
    print(f"  Session ID: {memory_config['session_id']}")
    print(f"  Agent ID: {memory_config['agent_id']}")
    print(f"  Username: {memory_config['username']}")
    print(f"  Endpoint: {memory_config['config'].db_endpoint}")
except Exception as e:
    print(f"ℹ Note: {type(e).__name__}")
    print(f"  In production, ensure AWS credentials and RDS instance exist")

# ========================================================================
# Example 2: Using IAM Authentication
# ========================================================================
print("\n" + "=" * 80)
print("Example 2: Oracle Agent Memory with IAM Authentication")
print("=" * 80)

try:
    memory_config = AWSOracleAgentMemoryFactory.create_from_iam_auth(
        db_instance_identifier="oracle-prod-db",
        db_endpoint="mydb.xxxxx.us-east-1.rds.amazonaws.com",
        db_user="iamuser",
        session_id="iam_session_001",
        agent_id="iam_agent",
        aws_region="us-east-1",
    )
    print("✓ Configuration created with IAM authentication")
    print(f"  Session ID: {memory_config['session_id']}")
    print(f"  Agent ID: {memory_config['agent_id']}")
    print(f"  IAM User: {memory_config['username']}")
    print(f"  Auth Token: {memory_config['auth_token'][:50]}...")
    print(f"  Token valid for: 15 minutes")
except Exception as e:
    print(f"ℹ Note: {type(e).__name__}")
    print(f"  IAM auth requires valid AWS credentials")

# ========================================================================
# Example 3: RDS Instance Information
# ========================================================================
print("\n" + "=" * 80)
print("Example 3: Retrieve RDS Instance Information")
print("=" * 80)

try:
    rds_config = AWSRDSOracleConfig(
        db_instance_identifier="oracle-prod-db",
        db_endpoint="mydb.xxxxx.us-east-1.rds.amazonaws.com",
        aws_region="us-east-1",
    )
    
    print("✓ RDS Configuration object created")
    print("  In production, call get_rds_instance_info() to retrieve:")
    print("  - Endpoint and port")
    print("  - Instance status")
    print("  - Engine version")
    print("  - Allocated storage")
    print("  - Instance class")
except Exception as e:
    print(f"ℹ Note: {e}")

print("\n" + "=" * 80)
print("✓ Configuration examples completed!")
print("=" * 80)

## 9. CloudWatch Monitoring Testing

Test CloudWatch monitoring capabilities for agent memory operations.

In [ ]:
print("\n" + "=" * 80)
print("CloudWatch Monitoring Example")
print("=" * 80)

# Create monitoring instance
monitoring = AWSMemoryMonitoring(aws_region="us-east-1")

# Log various metrics
print("\nLogging metrics to CloudWatch...")

monitoring.log_conversation_length(
    session_id="session_001",
    message_count=42,
    agent_id="chat_agent",
)
print("✓ Logged: ConversationLength = 42 messages")

monitoring.log_db_operation_time(
    "QueryExecution",
    duration_ms=245.5,
    session_id="session_001",
)
print("✓ Logged: QueryExecutionDuration = 245.5 ms")

monitoring.log_db_operation_time(
    "EntityExtraction",
    duration_ms=128.3,
    session_id="session_001",
)
print("✓ Logged: EntityExtractionDuration = 128.3 ms")

monitoring.log_db_operation_time(
    "VectorSearch",
    duration_ms=312.7,
    session_id="session_001",
)
print("✓ Logged: VectorSearchDuration = 312.7 ms")

# Display logged metrics
logged_metrics = monitoring.get_logged_metrics()
print(f"\nTotal metrics logged: {len(logged_metrics)}")
for metric in logged_metrics:
    print(f"  - {metric['MetricName']}: {metric['Value']} {metric['Unit']}")

print("\n" + "=" * 80)

## 10. AWS Lambda Handler Testing

Test the AWS Lambda function handler with sample events.

In [ ]:
print("\n" + "=" * 80)
print("AWS Lambda Handler Example")
print("=" * 80)

# Create Lambda handler
handler = AWSLambdaMemoryHandler(aws_region="us-east-1")
lambda_handler = handler.create_lambda_handler()

print("\n✓ Lambda handler created")

# Test with valid event
print("\nTest 1: Valid Lambda Event")
print("-" * 80)
test_event = {
    "body": {
        "message": "What is in my conversation history?",
        "session_id": "user-123",
        "agent_id": "chat_agent"
    }
}

response = lambda_handler(test_event, None)
print(f"Event: {json.dumps(test_event, indent=2)}")
print(f"\nResponse:")
print(json.dumps(json.loads(response['body']), indent=2))

# Test with missing fields
print("\n\nTest 2: Invalid Lambda Event (Missing Fields)")
print("-" * 80)
invalid_event = {
    "body": {
        "message": "Test message"
        # Missing session_id
    }
}

response = lambda_handler(invalid_event, None)
print(f"Event: {json.dumps(invalid_event, indent=2)}")
print(f"\nResponse:")
print(json.dumps(json.loads(response['body']), indent=2))

# Test with JSON string body
print("\n\nTest 3: Lambda Event with JSON String Body")
print("-" * 80)
test_event_json_body = {
    "body": json.dumps({
        "message": "Test message",
        "session_id": "user-456",
        "agent_id": "analysis_agent"
    })
}

response = lambda_handler(test_event_json_body, None)
print(f"Event (body is JSON string):")
print(json.dumps(test_event_json_body, indent=2))
print(f"\nResponse:")
print(json.dumps(json.loads(response['body']), indent=2))

print("\n" + "=" * 80)
print("Lambda handler examples completed!")
print("=" * 80)

## 11. Visualize Metrics and Performance

Create visualizations of CloudWatch metrics and performance data.

In [ ]:
# Create sample performance data for visualization
performance_data = {
    "operations": ["QueryExecution", "EntityExtraction", "VectorSearch", "MessageParsing", "ResponseGeneration"],
    "duration_ms": [245.5, 128.3, 312.7, 45.2, 789.4],
    "success_rate": [99.8, 99.5, 98.2, 99.9, 97.3],
}

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('AWS Oracle Agent Memory - Performance Metrics', fontsize=16, fontweight='bold')

# Plot 1: Operation Duration
ax1 = axes[0, 0]
colors_ops = plt.cm.viridis(np.linspace(0, 1, len(performance_data["operations"])))
bars = ax1.barh(performance_data["operations"], performance_data["duration_ms"], color=colors_ops)
ax1.set_xlabel('Duration (ms)', fontweight='bold')
ax1.set_title('Database Operation Durations', fontweight='bold')
for i, v in enumerate(performance_data["duration_ms"]):
    ax1.text(v + 20, i, f'{v:.1f}ms', va='center', fontweight='bold', fontsize=9)

# Plot 2: Success Rate
ax2 = axes[0, 1]
operations_short = [op[:15] + '...' if len(op) > 15 else op for op in performance_data["operations"]]
colors_success = ['#2ecc71' if x > 99 else '#f39c12' if x > 98 else '#e74c3c' for x in performance_data["success_rate"]]
ax2.bar(range(len(operations_short)), performance_data["success_rate"], color=colors_success)
ax2.set_xticks(range(len(operations_short)))
ax2.set_xticklabels(operations_short, rotation=45, ha='right')
ax2.set_ylabel('Success Rate (%)', fontweight='bold')
ax2.set_title('Operation Success Rate', fontweight='bold')
ax2.set_ylim([95, 100.5])
for i, v in enumerate(performance_data["success_rate"]):
    ax2.text(i, v + 0.2, f'{v:.1f}%', ha='center', fontweight='bold', fontsize=9)

# Plot 3: AWS Authentication Methods
ax3 = axes[1, 0]
auth_methods = ['Secrets\nManager', 'IAM\nAuth\nToken', 'Manual\nCredentials']
auth_usage = [45, 35, 20]
colors_auth = ['#3498db', '#9b59b6', '#e67e22']
wedges, texts, autotexts = ax3.pie(auth_usage, labels=auth_methods, autopct='%1.1f%%', 
                                     colors=colors_auth, startangle=90)
ax3.set_title('Authentication Method Distribution', fontweight='bold')
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# Plot 4: AWS Services Integration
ax4 = axes[1, 1]
ax4.axis('off')
services_text = """
AWS SERVICES INTEGRATION

✓ RDS Oracle Database
  Managed relational database service
  Automatic backups and scaling

✓ Secrets Manager
  Secure credential storage
  Automatic rotation support

✓ IAM Authentication
  Token-based access (15 min expiry)
  Enhanced security posture

✓ CloudWatch Monitoring
  Custom metrics logging
  Real-time dashboards

✓ Lambda Integration
  Serverless function support
  Event-driven processing

✓ VPC Support
  Private database connectivity
  Security group management
"""
ax4.text(0.05, 0.95, services_text, fontsize=10, family='monospace',
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='#ecf0f1', alpha=0.8))

plt.tight_layout()
plt.show()

print("✓ Performance metrics visualization displayed!")

---

## AWS Deployment Guides

### Deployment Patterns

#### 1. AWS Lambda Deployment

```python
# lambda_function.py
from oracle_agent_memory_aws import AWSLambdaMemoryHandler

handler_instance = AWSLambdaMemoryHandler(aws_region="us-east-1")
lambda_handler = handler_instance.create_lambda_handler()

# Set environment variables in Lambda:
# - AWS_REGION
# - RDS_ORACLE_ENDPOINT
# - RDS_ORACLE_INSTANCE_ID
# - RDS_DB_SECRET_NAME or RDS_DB_IAM_USER
```

#### 2. AWS ECS Deployment

```dockerfile
FROM python:3.11-slim

WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .

CMD ["python", "app.py"]
```

Set environment variables in ECS task definition.

#### 3. AWS EC2 Deployment

1. Launch EC2 instance in same VPC as RDS
2. Install Python and dependencies
3. Configure security groups for database access
4. Run application with environment variables

### Environment Variable Configuration

#### For Secrets Manager Authentication

```bash
export AWS_REGION=us-east-1
export RDS_ORACLE_ENDPOINT=mydb.xxxxx.us-east-1.rds.amazonaws.com
export RDS_ORACLE_INSTANCE_ID=oracle-prod-db
export RDS_ORACLE_PORT=1521
export RDS_ORACLE_DB_NAME=ORCL
export RDS_DB_SECRET_NAME=rds/oracle/credentials
```

#### For IAM Database Authentication

```bash
export AWS_REGION=us-east-1
export RDS_ORACLE_ENDPOINT=mydb.xxxxx.us-east-1.rds.amazonaws.com
export RDS_ORACLE_INSTANCE_ID=oracle-prod-db
export RDS_DB_IAM_USER=iamuser
```

### Security Best Practices

1. **Secrets Manager** - Store credentials securely, never in code
2. **IAM Authentication** - Use token-based auth for enhanced security
3. **VPC** - Deploy RDS and application in same VPC
4. **Security Groups** - Restrict database access to application IPs
5. **IAM Roles** - Use instance/task roles instead of access keys
6. **Encryption** - Enable RDS encryption at rest
7. **Monitoring** - Use CloudWatch for real-time alerts

### Performance Optimization

| Setting | Recommendation |
|---------|-----------------|
| Connection Pooling | Use SQLAlchemy with pool_size=10 |
| RDS Instance Class | db.r6g.xlarge for production |
| Storage | io1 with 1000 IOPS for high throughput |
| Backup | Automated daily backups + multi-AZ |
| Cache | ElastiCache for frequently accessed data |

### Troubleshooting

**Issue**: "Unable to locate credentials"
- **Solution**: Configure AWS credentials via IAM role or credentials file

**Issue**: "Connection refused to RDS"
- **Solution**: Check security group allows traffic from application

**Issue**: "IAM token expired"
- **Solution**: Token expires after 15 minutes; request new token automatically

**Issue**: "Secrets Manager secret not found"
- **Solution**: Verify secret name and IAM permissions

---

## Summary and Key Features

### Core AWS Features Implemented

| Feature | Capability | Use Case |
|---------|-----------|----------|
| **RDS Instance Discovery** | Query RDS metadata | Auto-scaling and monitoring |
| **Secrets Manager** | Secure credential retrieval | Production credentials management |
| **IAM Auth Tokens** | 15-minute expiring tokens | Enhanced security posture |
| **CloudWatch Metrics** | Custom metric logging | Real-time monitoring and alerts |
| **Lambda Handler** | Serverless function integration | Event-driven processing |
| **Environment Config** | Auto-detection from env vars | DevOps-friendly deployment |

### AWS Services Integrated

- **Amazon RDS** - Managed Oracle database
- **AWS Secrets Manager** - Credential management
- **AWS IAM** - Identity and access management
- **Amazon CloudWatch** - Monitoring and logging
- **AWS Lambda** - Serverless computing
- **AWS ECS** - Container orchestration
- **Amazon EC2** - Compute instances
- **Amazon VPC** - Private networking

### Key Classes

- `AWSRDSOracleConfig` - RDS configuration management
- `AWSEnvironmentConfig` - Environment variable loader
- `AWSOracleAgentMemoryFactory` - Memory instance factory
- `AWSMemoryMonitoring` - CloudWatch integration
- `AWSLambdaMemoryHandler` - Lambda function handler

### Quick Start

1. **Set up AWS RDS Oracle**
   - Create RDS instance in VPC
   - Configure security groups

2. **Store credentials**
   - Use Secrets Manager for production
   - Or enable IAM database authentication

3. **Configure environment**
   - Set AWS_REGION, RDS endpoints
   - Set authentication credentials/IAM user

4. **Deploy application**
   - Lambda: Use lambda_handler function
   - ECS: Docker container with env vars
   - EC2: Python app with IAM role

5. **Monitor with CloudWatch**
   - View custom metrics
   - Set up alarms for performance

### Deployment Checklist

- [ ] RDS Oracle instance created and running
- [ ] VPC and security groups configured
- [ ] Credentials stored in Secrets Manager
- [ ] IAM roles created with proper permissions
- [ ] Application deployed with environment variables
- [ ] CloudWatch dashboards created
- [ ] Alarms configured for critical metrics
- [ ] Backup and disaster recovery tested

### Additional Resources

- [AWS RDS Oracle Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/CHAP_Oracle.html)
- [AWS Secrets Manager Guide](https://docs.aws.amazon.com/secretsmanager/)
- [RDS Database Authentication](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/UsingWithRDS.IAMDBAuth.html)
- [CloudWatch Monitoring](https://docs.aws.amazon.com/cloudwatch/)
- [Lambda Best Practices](https://docs.aws.amazon.com/lambda/latest/dg/best-practices.html)